<a href="https://colab.research.google.com/github/itu-itis23-majidov23/Prediction-of-Coordinates-based-on-Similarity/blob/main/CLIP_and_Cosine_Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-mas_2pyc
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-mas_2pyc
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=0a9075f42aa332e49ecf4b6c26332a1f48724ded5874cd2214f0034d12cec17e
  Stored in directory: /tmp/pip-ephem-wheel-cache-bs5xuy5n/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from PIL import Image
import clip

# Load the CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

def load_image(image_path):
    input_image = Image.open(image_path)
    return input_image

def get_image_embeddings(object_image):
    # Preprocess the image and convert to tensor
    image_input = preprocess(object_image).unsqueeze(0).to(device)

    # Extract features using CLIP
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)

    return image_features.cpu().numpy()

def calculate_lat_lon_for_test_images(train_csv, train_folder, test_folder, output_csv):
    # Load training dataset
    train_data = pd.read_csv(train_csv, delimiter=';')

    # Compute embeddings for all training images
    train_embeddings = []
    for filename in train_data['filename']:
        image_path = f"{train_folder}/{filename}"
        image_obj = load_image(image_path)
        embedding = get_image_embeddings(image_obj)
        train_embeddings.append(embedding.flatten())
    train_embeddings = np.array(train_embeddings)

    # Get test image embeddings and predict latitude/longitude
    test_filenames = [file for file in os.listdir(test_folder) if file.endswith(('jpeg', 'jpg', 'png'))]
    results = []

    for test_filename in test_filenames:
        test_image_path = f"{test_folder}/{test_filename}"
        test_image = load_image(test_image_path)
        test_embedding = get_image_embeddings(test_image).flatten()

        # Calculate similarity with all training images
        similarities = cosine_similarity([test_embedding], train_embeddings)[0]

        # Get the indices of 4 most similar images
        top_indices = similarities.argsort()[-4:][::-1]

        # Calculate mean latitude and longitude
        top_latitudes = train_data.iloc[top_indices]['latitude']
        top_longitudes = train_data.iloc[top_indices]['longitude']

        predicted_latitude = top_latitudes.mean()
        predicted_longitude = top_longitudes.mean()

        # Append results
        results.append({
            'filename': test_filename,
            'latitude': predicted_latitude,
            'longitude': predicted_longitude
        })

    # Save results to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv, index=False)
    print(f"Results saved to {output_csv}")

# Example usage
train_csv = '/content/drive/MyDrive/Folder/Extracted/train.csv'  # CSV file containing filename, latitude, longitude
train_folder = '/content/drive/MyDrive/Folder/Extracted/train'  # Folder containing training images
test_folder = '/content/drive/MyDrive/Folder/Extracted/test'  # Folder containing test images
output_csv = '/content/submission_123.csv'  # Output CSV file

calculate_lat_lon_for_test_images(train_csv, train_folder, test_folder, output_csv)


100%|███████████████████████████████████████| 338M/338M [00:07<00:00, 45.7MiB/s]


Results saved to /content/submission_123.csv
